<a href="https://colab.research.google.com/github/derekbt96/ENPM673/blob/master/proj6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Copy of dogs-vs-cats.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
zip_ref = zipfile.ZipFile("/tmp/train.zip", 'r')
zip_ref.extractall("/content/dataset/")
zip_ref.close()

In [6]:
!mkdir /content/train_renamed

mkdir: cannot create directory ‘/content/train_renamed’: File exists


In [7]:
import cv2
import numpy as np
import os
import math

snc = 0
snd = 12500
# PATH = './train'
# for file in sorted(os.listdir(PATH)):
#     filepath = os.path.join(PATH,file)
#     if filepath.endswith(".jpg") or filepath.endswith(".pgm") or filepath.endswith(".png") or filepath.endswith(".ppm"):
for subdir, dirs, files in os.walk('/content/dataset/train'):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg") or filepath.endswith(".pgm") or filepath.endswith(".png") or filepath.endswith(".ppm"):
            # print(filepath)
                img = cv2.imread(filepath)
                img1 = img.copy()

                if "cat" in filepath:
                    print("Filepath {} has cat".format(filepath))
                    sss = "/content/train_renamed/{0}.png".format(snc)
                    cv2.imwrite(sss,img)
                    snc = snc+1

                elif "dog" in filepath:
                    print("Filepath {} has dog".format(filepath))
                    sss = "/content/train_renamed/{0}.png".format(snd)
                    cv2.imwrite(sss,img)
                    snd = snd+1


print(snc, snd)


Streaming output truncated to the last 5000 lines.
Filepath /content/dataset/train/dog.3752.jpg has dog
Filepath /content/dataset/train/dog.10208.jpg has dog
Filepath /content/dataset/train/dog.284.jpg has dog
Filepath /content/dataset/train/cat.2333.jpg has cat
Filepath /content/dataset/train/dog.2359.jpg has dog
Filepath /content/dataset/train/dog.7869.jpg has dog
Filepath /content/dataset/train/cat.9642.jpg has cat
Filepath /content/dataset/train/cat.6032.jpg has cat
Filepath /content/dataset/train/dog.834.jpg has dog
Filepath /content/dataset/train/dog.884.jpg has dog
Filepath /content/dataset/train/cat.109.jpg has cat
Filepath /content/dataset/train/cat.3586.jpg has cat
Filepath /content/dataset/train/dog.5595.jpg has dog
Filepath /content/dataset/train/dog.5238.jpg has dog
Filepath /content/dataset/train/cat.4524.jpg has cat
Filepath /content/dataset/train/dog.2240.jpg has dog
Filepath /content/dataset/train/dog.11088.jpg has dog
Filepath /content/dataset/train/dog.1181.jpg has d

In [8]:
!pip3 install cnn_finetune

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for cnn-finetune: filename=cnn_finetune-0.6.0-cp36-none-any.whl size=11432 sha256=3104033138eb4988d9eb1c37446e932c2145351dea999a571e45821ca02b3ee0
  Stored in directory: /root/.cache/pip/wheels/12/4f/29/32978e72b606c5f51b6bd10117814deb46c6b63230eef49381
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=de393d05932b217d5100bad546a4b60fa6a37ed37d0efa4ad28e8a974d9f96af
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built cnn-finetune pretrainedmodels


In [0]:
import numpy as np
import csv

with open('train_all.csv', mode='w+') as train_file:
    writer = csv.writer(train_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    i = 0
    class_label = 0

    for i in range(0,25000):

        if i>12499:
            class_label = 1

        writer.writerow(['{}.png'.format(i),'{}'.format(class_label)])


In [0]:
from __future__ import print_function, division

import matplotlib
# matplotlib.use('agg')

import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
from skimage.transform import rescale, resize
import torch.nn.functional as F     
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import warnings
import math
import sys

import numpy as np
import torchvision
from torchvision import datasets, models, transforms, utils
import time
import os
from os import path
from cnn_finetune import make_model

import random
import time

warnings.filterwarnings("ignore")

# parser = argparse.ArgumentParser(description='cnn_finetune')

# parser.add_argument('--batch-size', type=int, default=2, metavar='N',
#                     help='input batch size for training (default: 32)')

# parser.add_argument('--test-batch-size', type=int, default=4, metavar='N',
#                     help='input batch size for testing (default: 64)')

# parser.add_argument('--epochs', type=int, default=200, metavar='N',
#                     help='number of epochs to train (default: 100)')

# parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                     help='learning rate (default: 0.01)')


PATH = 'resnet50.pt'

thresh = 0.05

rdeftrain = 0
rdeftest = 0

beginning = time.time()

epno = 0
use_gpu=0

tta = []
toa = []

# args = parser.parse_args()

if torch.cuda.is_available():
    device = torch.device("cuda")
    use_gpu=1 
    print ("CUDA Available")

epochs = 4
lr = 0.7

batch_size = 64
batch_size_test = 64
epochs_num = epochs

print ("Batch size is {0}".format(batch_size))
print ("Batch size test is {0}".format(batch_size_test))
print ("Epoch size is {0}".format(epochs_num))
print ("This is nn7. name is 1")

model = make_model('resnet50', num_classes=2, pretrained=False, input_size=(224, 224))


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    # transforms.Normalize(
    #     mean=model.original_model_info.mean,
    #     std=model.original_model_info.std),
    # transforms.ToPILImage(),
])


# frun = open("values_running.csv","w+")

class Dataloader(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0])

        image = io.imread(img_name)

        # print(img_name)

        # image = resize(image,(250,250), anti_aliasing=True)

        # image = torchvision.transforms.ToPILImage(image)

        # image = torchvision.transforms.functional.to_pil_image(image)
        # image = torchvision.transforms.functional.resize(image,(225,225))

        # # image = torchvision.transforms.ToTensor(image)
        # image = torchvision.transforms.functional.to_tensor(image)

        # image = image.reshape([image.shape[2], image.shape[1], image.shape[0]])

        # image = torch.FloatTensor(image)

        class_label = self.data.iloc[idx, 1]

        if class_label==0:
            class_label = np.array([0])
        else:
            class_label = np.array([1])

        class_label = class_label.astype('long').reshape(1, 1)

        class_label = torch.LongTensor(class_label)

        sample = {'image': image, 'class_label': class_label}

        if self.transform:
            image1 = self.transform(sample['image'])
            image1 = torch.FloatTensor(image1)
            sample = {'image': image1, 'class_label': class_label}

        # sample = data_utils.TensorDataset(sample)

        return sample
#################################################

# class_label_frame = pd.read_csv("/users/v.dorbala/me/vpscmu.csv")

# n = 65
# img_name = class_label_frame.iloc[n, 0]
# class_label = class_label_frame.iloc[n, 1:].as_matrix()
# class_label = class_label.astype('float').reshape(-1, 1)

# print('Image name: {}'.format(img_name))
# print('class_label shape: {}'.format(class_label.shape))
# print('class_label value: {}'.format(class_label[:1]))
# model_conv = torchvision.models.alexnet(pretrained=True)

# num_features = 1

# num_ftrs = model_conv.classifier[6].in_features
# model_conv.classifier[6] = nn.Linear(num_ftrs, num_features)

# for param in model_conv.classifier[6].parameters():
#   param.requires_grad = True

# print(model)

model_ft = model.to(device)

if use_gpu == 1:
    model_ft = nn.DataParallel(model_ft).cuda()

if path.exists(PATH):
    model_ft.load_state_dict(torch.load(PATH))

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9, weight_decay=0.005)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

trainset = Dataloader(csv_file='/content/train_all.csv', root_dir='/content/train_renamed',transform=transform)

# testset = Dataloader(csv_file='test_all.csv', root_dir='./train_renamed',transform=transform)
# print ("Size of the dataset is {}".format(len(trainset)))
#transforms.Compose([Rescale(256),RandomCrop(224),ToTensor()]))

# trsize = int(len(trainset)*2/3)
# tesize = len(trainset) - trsize
# print (trsize,tesize,len(trainset))

validation_split = 0.1
shuffle_dataset = True

nums = [x for x in range(10,100)]
random.shuffle(nums)

random_seed=nums[int(time.time())%90]

# Creating data indices for training and validation splits:
dataset_size = len(trainset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

# trainset, testset = torch.utils.data.random_split(trainset,[trsize,tesize])
# print (len(trainset),len(testset))
# train_sampler = SubsetRandomSampler(trainset)
# test_sampler = SubsetRandomSampler(testset)

# print (type(train_sampler))

# trainset = Dataloader(csv_file='trainall.csv', root_dir='/users/v.dorbala/me/fin3/',transform=transform)#transforms.Compose([Rescale(256),RandomCrop(224),ToTensor()]))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          num_workers=2,sampler=train_sampler)
# testset = Dataloader(csv_file='vps1.csv', root_dir='/users/v.dorbala/me/testfin/',transform=transform)#transforms.Compose([Rescale(256),RandomCrop(224),ToTensor()]))

test_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_test,
                                        num_workers=2,sampler=valid_sampler)


# Helper function to show a batch
# def show_landmarks_batch(sample_batched):
#     """Show image with landmarks for a batch of samples."""
#     images_batch, landmarks_batch = \
#             sample_batched['image'], sample_batched['class_label']
#     batch_size = len(images_batch)
#     im_size = images_batch.size(2)

#     grid = utils.make_grid(images_batch)
#     plt.imshow(grid.numpy().transpose((1, 2, 0)))

#     # for i in range(batch_size):
#     #     plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size,
#     #                 landmarks_batch[i, :, 1].numpy(),
#     #                 s=10, marker='.', c='r')

# for i_batch, sample_batched in enumerate(train_loader):
#     print(i_batch, sample_batched['image'].size(),
#           sample_batched['class_label'].size())

#     # observe 4th batch and stop.
#     if i_batch == 3:
#         plt.figure()
#         show_landmarks_batch(sample_batched)
#         plt.axis('off')
#         plt.title('Train Dataset Batch')
#         plt.ioff()
#         plt.show()
#         break


# for i_batch, sample_batched in enumerate(test_loader):
#     print(i_batch, sample_batched['image'].size(),
#           sample_batched['class_label'].size())

#     # observe 4th batch and stop.
#     if i_batch == 3:
#         plt.figure()
#         show_landmarks_batch(sample_batched)
#         plt.axis('off')
#         plt.ioff()
#         plt.title('Test Dataset Batch')
#         plt.show()
#         break
# testset = torchvision.datasets.CIFAR10(root='./Cifar10', train=False,
#                                        download=True, transform=transform)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

trainlossarr = []
testlossarr = []

trainrsqarr = []
testrsqarr = []
# def restart_program():

#     os.execl(sys.executable, os.path.abspath(__file__), *sys.argv)
#     python = sys.executable
#     os.execl(python, python, * sys.argv)


def train(epoch):
    global rdeftrain
    since = time.time()
    total_loss = 0
    total_size = 0
    traintargetarr = []
    trainoutputarr = []
    model_ft.train()

    print ("Epoch number is {}".format(epoch))

    for batch_idx, values in enumerate(train_loader):
        
        # print("Values are size {0}, {1}, {2}".format(len(values),(values['image'].shape),(values['class_label'].shape)))

        data, target = values['image'], values['class_label']

        # print("Data type is {0}, {1}.".format(data,data.shape))

        target = target.view(-1,1)

        # print("Now target is {0}".format(target)) 

        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        output = model_ft(data)

        # print ("Target is {0}. \n Output is {1}.".format(target.numpy(),output.numpy()))

        # print(batch_idx)

        # rsq = r2_score(target.numpy(),output.numpy())
        
        traintargetarr.append(target.tolist())
        trainoutputarr.append(output.tolist())


        loss = criterion(output, target.squeeze())

        total_loss += loss.item()

        total_size += 1

        # print ("Data size is {}. \n Total size is {}".format(data.size(0),total_size))

        
        loss.backward()
        
        optimizer.step()

        # if batch_idx % args.log_interval == 10:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader), total_loss / total_size))
        # if math.isnan(total_loss) == True:
        #     sys.exit('Loss has gone to NaN. Doubling the batch size.')
            # restart_program()

        print((total_loss/total_size)*100)

    tta = np.array(traintargetarr)
    tta = [val for sublist in traintargetarr for val in sublist]
    tta = np.array(tta)
    toa = np.array(trainoutputarr)
    toa = [val for sublist in trainoutputarr for val in sublist]
    toa = np.array(toa)
    # print (tta.flatten(),toa.flatten())
    # rsqtrain = r2_score(tta.flatten(),toa.flatten())

    # vconst = 0.2
    # pinv = (-1)*(np.linalg.pinv(Jw))
    # fmat = le + Jv*vconst
    # w = pinv*(fmat)

    # print ("RSQtrain is {}".format(rsqtrain))
    trainlossarr.append((total_loss/total_size))
    # trainrsqarr.append(rsqtrain)
    # if rsqtrain>rdeftrain:
    #     rdeftrain = rsqtrain
    #     # model_ft_best = copy.deepcopy(model_ft)
    #     torch.save(model_ft.state_dict(), PATH)
    # frun.write("{},".format(total_loss/total_size))
    # time_elapsed = time.time() - since
    # print('Training complete in {:.0f}m {:.0f}s'.format(
    #     time_elapsed // 60, time_elapsed % 60))


def test():
    global tta,toa,rdeftest
    # model_ft.load_state_dict(torch.load(PATH))
    model_ft.eval()
    test_loss = 0
    total_loss = 0
    total_size = 0

    testtargetarr = []
    testoutputarr = []

    with torch.no_grad():
        for batch_idx, values in enumerate(test_loader):

            data, target = values['image'], values['class_label']

            target = target.view(-1,1)

            data, target = data.to(device), target.to(device)

            # print (data.size())

            output = model_ft(data)

            # print ("Test Target is {0}. \n Output is {1}.".format(target,output))
            # print ("Target type is {0}. \n Output type is {1}.".format(target.type,output.type))
            # print ("Target shape is {0}. \n Output shape is {1}.".format(target.shape,output.shape))

            loss = criterion(output, target.squeeze())

            total_loss += loss.item()

            total_size += 1
        
            testtargetarr.append(target.tolist())
            testoutputarr.append(output.tolist())

            print ("Loss is {}".format(loss*100))
            # pred = output.data.max(1, keepdim=True)[0]

            # correct += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    tta = np.array(testtargetarr)
    tta = [val for sublist in testtargetarr for val in sublist]
    tta = np.array(tta)
    toa = np.array(testoutputarr)
    toa = [val for sublist in testoutputarr for val in sublist]
    toa = np.array(toa)

    # rsqtest = r2_score(tta.flatten(),toa.flatten())
    # print ("RSQtest is {}\n".format(rsqtest))

    # test_loss /= len(test_loader.dataset)    
    testlossarr.append(total_loss/total_size)
    # testrsqarr.append(rsqtest)
    # frun.write("{},".format(test_loss))



for epoch in range(1, epochs + 1):
    train(epoch)
    test()
    print ("\n Epoch number is {}.".format(epoch))

    if epoch % 10 == 0:
        print("Loss at epoch {} is {}%".format(epoch,100*(fincorr/epoch)))

# torch.save(model_ft.state_dict(), PATH)

time_elapsed = time.time() - beginning
print('Process complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))


# trace = dict(x=[1:args.epochs], y=testlossarr, mode="markers+lines", type='custom',
#              marker={'color': 'red', 'symbol': 104, 'size': "10"}, name='1st Trace')

trainlossarr = np.array(trainlossarr)
testlossarr = np.array(testlossarr)

trainrsqarr = np.array(trainrsqarr)
testrsqarr = np.array(testrsqarr)

file = open("resnet50.csv","w+")

file.write("Train values are \n")

for i in range(len(trainlossarr)):
    file.write("{},".format(trainlossarr[i]))

file.write("\n Test values are \n")

for i in range(len(testlossarr)):
    file.write("{},".format(testlossarr[i]))

# file.write("\n Train R-squared values are \n")

# for i in range(len(trainrsqarr)):
#     file.write("{},".format(trainrsqarr[i]))

# file.write("\n Test R-squared values are \n")

# for i in range(len(testrsqarr)):
#     file.write("{},".format(testrsqarr[i]))

file.close()

t = np.arange(0,epochs_num,1)

plt.plot(t,trainlossarr,t,testlossarr)
plt.xlabel("Epochs")
plt.ylabel("Loss value")
plt.title("Train and test losses (Normal)")
plt.show()
# print ("Overall Loss on test data is {}%)".format(test_loss))


# viz.line(X=None, Y=None, win=win, name='delete this', update='remove')


# layout = dict(title="Testing Loss", xaxis={'Epoch': 'x1'}, yaxis={'Loss': 'x2'})

# vis._send({'data': win, 'layout': layout, 'win': 'mywin'})
# resize = [224,224]    
# data_transforms = {
#         'test': transforms.Compose([
#             #Higher scale-up for inception
#             transforms.Resize(max(resize)),
#             #transforms.RandomHorizontalFlip(),
#             #transforms.CenterCrop(max(resize)),
#             transforms.ToTensor(),
#             transforms.Normalize([0.620, 0.446, 0.594], [0.218, 0.248, 0.193])
#         ]),
#     }

# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'test']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'test']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

# class_names = image_datasets['train'].classes

# def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'test']:
#           print (phase)
#             if phase == 'train':
#                 scheduler.step()
#                 model.train()  # Set model to training mode
#             else:
#                 model.test()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # zero the parameter gradients
#                 optimizer.zero_grad()

#                 # forward
#                 # track history if only in train
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     # backward + optimize only if in training phase
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 # statistics
#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             epoch_loss = running_loss / dataset_sizes[phase]
#             epoch_acc = running_corrects.double() / dataset_sizes[phase]

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=25)

CUDA Available
Batch size is 64
Batch size test is 64
Epoch size is 4
This is nn7. name is 1
Epoch number is 1
70.23606896400452
1696.3376313447952
10831.033152341843
12168.153251707554
10109.398738145828
8533.599735299747
7364.851594822747
6514.144317060709
5804.505567418205
5231.653224229813
4797.228874943473
4415.772919853529
4081.9144615760215
3811.186812605177
3563.8572859764095
3346.0109166800976
3167.2688379007227
3001.3579421573218
2847.467040074499
2714.7353413701057
2588.664898134413
2478.799632191658
2375.9771603605022
2280.2131046851473
2193.4260358810425
2111.6697258674185
2035.9911697882194
1968.533687080656
1902.937983849953
1841.7349265019097
1785.9741962725118
1733.5843870416284
1683.2876987529523
1635.7628327958726
1591.793601172311
1549.4417127635745
1510.3773216943484
1472.816020564029
1437.1028108474536
1403.4766310453415
1370.925790362242
1341.3612557309013
1312.4495586683583
1284.4530796462839
1257.6674649450513
1232.173005523889
1207.44159234331
1183.70961571733